### Clean data
Ce notebook sert à charger et nétoyer les données

In [1]:
import pandas as pd
import numpy as np

In [2]:
path = "../data/raw/top_100_saas_companies_2025.csv"
df = pd.read_csv(path)

In [5]:
df.head()

,Unnamed: 0,Company Name,Founded Year,HQ,Industry,Total Funding,ARR,Valuation,Employees,Top Investors,Product,G2 Rating
0,0,Microsoft,1975,"Redmond, WA, USA",Enterprise Software,$1B,$270B,$3T,"221,000","Bill Gates, Paul Allen","Azure, Office 365, Teams",4.4
1,1,Salesforce,1999,"San Francisco, CA, USA",CRM,$65.4M,$37.9B,$227.8B,"75,000","Halsey Minor, Larry Ellison","Sales Cloud, Service Cloud",4.3
2,2,Adobe,1982,"San Jose, CA, USA",Creative Software,$2.5M,$19.4B,$240B,"29,945",Hambrecht & Quist,"Creative Cloud, Document Cloud",4.5
3,3,Oracle,1977,"Austin, TX, USA",Database & Enterprise,$2K,$52.9B,$350B,"143,000","Larry Ellison, Bob Miner","Oracle Cloud, NetSuite",4.0
4,4,SAP,1972,"Walldorf, Germany",Enterprise Software,NaN,$32.5B,$215B,"107,415","Dietmar Hopp, Klaus Tschira","S/4HANA, SuccessFactors",4.1


In [6]:
df = df.drop(columns=["Unnamed: 0"])
renamed_columns = {    "Company Name": "nom_entreprise",
            "Founded Year": "annee_creation",
            "HQ": "QG",
            "Industry": "secteur",
            "Total Funding": "capital",
            "ARR": "ca",
            "Valuation": "valorisation",
            "Employees": "nombre_employes",
            "Top Investors": "top_investisseurs",
            "Product": "produit",
            "G2 Rating": "cote_g2",
        }

df = df.rename(columns= renamed_columns)

Pas d'info sur le capital de SAP. Supprimer pour le moment.

In [8]:
df = df.dropna()

## Transformations

In [9]:
def tranform_column_capital(colonne: np.ndarray) -> np.ndarray:
    transformed_values = []
    
    for v in colonne:
        v = v[1:]
        num_part = ""
        unit = None
        
        for c in v:
            if c.isdigit() or c == ".": 
                num_part += c
            else:
                unit = c
                break

       
        if not num_part:  
            # num_part = np.nan
            print("Cette valeur n'est pas convertie")
            break
        else:
            try:
                if "." in num_part:
                    num_part = num_part.replace(".", "")
                num_part = float(num_part) 
                if unit == "B":
                    num_part *= 1_000_000_000
                elif unit == "M":
                    num_part *= 1_000_000
            except ValueError:
                num_part = np.nan 
        
        transformed_values.append(num_part)

    return np.array(transformed_values)

def transform_column_ca(colonne: np.ndarray) -> np.ndarray:
    transformed_ca = []
    
    for v in colonne:
        v=str(v)
        v = v.lstrip("$")  
        
        if v.endswith("T"):
            v = float(v[:-1]) * 1_000_000_000_000
        elif v.endswith("B"):
            v = float(v[:-1]) * 1_000_000_000
        elif v.endswith("M"):
            v = float(v[:-1]) * 1_000_000
        else:
            try:
                v = float(v)  
            except ValueError:
                # v = np.nan
                print("Une des valeur contient des charactères non numériques")
                raise
        
        transformed_ca.append(v)
    
    return np.array(transformed_ca)

def transform_column_valorisation(colonne: np.ndarray) -> np.ndarray:
    transformed_values = []
    
    for v in colonne:
        v=str(v)
        v = v.lstrip("$").split(" (")[0]
        
        unit = None
        if "T" in v:
            index_T = v.index("T")
            num_part = v[:index_T]
            unit = "T"
        elif "B" in v:
            index_B = v.index("B")
            num_part = v[:index_B]
            unit = "B"
        elif "M" in v:
            index_M = v.index("M")
            num_part = v[:index_M]
            unit = "M"
        else:
            num_part = v  
        
        try:
            num_part = float(num_part)
            if unit == "T":
                num_part *= 1_000_000_000_000
            elif unit == "B":
                num_part *= 1_000_000_000
            elif unit == "M":
                num_part *= 1_000_000
        except ValueError:
            # num_part = np.nan 
            print("Une des valeur contient des charactères non numériques")
            raise
        
        transformed_values.append(num_part)

    return np.array(transformed_values)

def tranform_column_nombre_employes(colonne: pd.Series) -> pd.Series:
    colonne = colonne.astype(str).apply(lambda v: v.replace(',', '')).astype(float)
    return colonne



In [10]:
df["capital_$"] = tranform_column_capital(df["capital"])
df["ca_$"] = transform_column_ca(df["ca"].to_numpy())
df["valorisation_$"] = transform_column_valorisation(df["valorisation"].to_numpy())
df["nombre_employes"] = tranform_column_nombre_employes(df["nombre_employes"])


df.drop(columns=["capital"], inplace=True)
df.drop(columns=["ca"], inplace=True)
df.drop(columns=["valorisation"], inplace=True)

C:\Users\serig\AppData\Local\Temp\ipykernel_23932\3776767352.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["capital_$"] = tranform_column_capital(df["capital"])
C:\Users\serig\AppData\Local\Temp\ipykernel_23932\3776767352.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ca_$"] = transform_column_ca(df["ca"].to_numpy())
C:\Users\serig\AppData\Local\Temp\ipykernel_23932\3776767352.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

In [14]:
# Sauvegarder les données propres dans data/cleaned_data
df.to_csv("../data/cleaned_data/top_100_startup_2025.csv", index=False)